In [4]:
# TEMP: Import lagom
# Not useful once lagom is installed
import sys
sys.path.append('/home/zuoxingdong/Documents/Code/lagom/')

In [18]:
from lagom.envs.spaces import Box, Dict, Discrete

s = Dict({'position': Discrete(2), 'velocity': Discrete(3)})
s.spaces.values()

odict_values([Discrete(2), Discrete(3)])

In [24]:
PartialFlattenDict

numpy.int64

In [ ]:
class PartialFlattenDictWrapper(gym.ObservationWrapper):
    def __init__(self, env, dict_keys):

        # Figure out observation_space dimension.
        size = 0
        for key in dict_keys:
            shape = self.env.observation_space.spaces[key].shape
            size += np.prod(shape)
        self.observation_space = gym.spaces.Box(-np.inf, np.inf, shape=(size,), dtype='float32')

    def observation(self, observation):
        assert isinstance(observation, dict)
        obs = []
        for key in self.dict_keys:
            obs.append(observation[key].ravel())
        return np.concatenate(obs)

In [ ]:
class VecEnv(object):
    """
    An asynchronous, vectorized environment. 
    """
    def __init__(self, num_envs, observation_space, action_space):
        self.num_envs = num_envs
        self.observation_space = observation_space
        self.action_space = action_space
       
    def step_async(self, actions):
        """
        All environments take a step with the given actions. 
        
        Call step_wait() to obtain the outputs. 
        
        Note: Do not call this function when a step_async is pending
        """
        raise NotImplementedError
        
    def step_wait(self):
        """
        Wait for step_async(). 
        
        Returns:
            observations (array): single or a tuple of observations
            rewards (array): rewards
            dones (array): booleans of episode terminations
            infos (array): info objects
        """
        raise NotImplementedError
        
    def step(self, actions):
        self.step_async(actions)
        
        return self.step_wait()
    
    def reset(self):
        """
        Reset all environments and returns a tuple of observations. 
        
        step_async() will be cancelled. 
        """
        raise NotImplementedError
        
    def render(self):
        pass
        
    def close(self):
        """
        Closing the environments
        """
        raise NotImplementedError
        
    @property
    def observation_space(self):
        """
        Return a Space object to define the observation space.
        """
        return self.observation_space
    
    @property
    def action_space(self):
        """
        Return a Space object to define the action space.
        """
        return self.action_space

In [ ]:
class DummyVecEnv(VecEnv):
    def __init__(self, list_make_env):
        self.envs = [make_env() for make_env in list_make_env]
        observation_space = self.envs[0].observation_space
        action_space = self.envs[0].action_space
        super().__init__(len(list_make_env), observation_space, action_space)
        
        

In [ ]:
class DummyVecEnv(VecEnv):
    def __init__(self, env_fns):
        
        obs_spaces = self.observation_space.spaces if isinstance(self.observation_space, gym.spaces.Tuple) else (self.observation_space,)
        self.buf_obs = [np.zeros((self.num_envs,) + tuple(s.shape), s.dtype) for s in obs_spaces]
        self.buf_dones = np.zeros((self.num_envs,), dtype=np.bool)
        self.buf_rews  = np.zeros((self.num_envs,), dtype=np.float32)
        self.buf_infos = [{} for _ in range(self.num_envs)]
        self.actions = None

    def step_async(self, actions):
        self.actions = actions

    def step_wait(self):
        for i in range(self.num_envs):
            obs_tuple, self.buf_rews[i], self.buf_dones[i], self.buf_infos[i] = self.envs[i].step(self.actions[i])
            if self.buf_dones[i]:
                obs_tuple = self.envs[i].reset()
            if isinstance(obs_tuple, (tuple, list)):
                for t,x in enumerate(obs_tuple):
                    self.buf_obs[t][i] = x
            else:
                self.buf_obs[0][i] = obs_tuple
        return (self._obs_from_buf(), np.copy(self.buf_rews), np.copy(self.buf_dones),
                self.buf_infos.copy())

    def reset(self):
        for i in range(self.num_envs):
            obs_tuple = self.envs[i].reset()
            if isinstance(obs_tuple, (tuple, list)):
                for t,x in enumerate(obs_tuple):
                    self.buf_obs[t][i] = x
            else:
                self.buf_obs[0][i] = obs_tuple
        return self._obs_from_buf()

    def close(self):
        return

    def _obs_from_buf(self):
        if len(self.buf_obs) == 1:
            return np.copy(self.buf_obs[0])
        else:
            return tuple(np.copy(x) for x in self.buf_obs)

In [ ]:
import numpy as np
from multiprocessing import Process  # easier code than threading
from multiprocessing import Pipe  # Much faster than Queue


class CloudpickleWrapper(object):
    """
    Uses cloudpickle to serialize contents (otherwise multiprocessing tries to use pickle)
    """
    def __init__(self, x):
        self.x = x
        
    def __call__(self):
        return self.x()
    
    def __getstate__(self):
        import cloudpickle
        return cloudpickle.dumps(self.x)
    def __setstate__(self, ob):
        import pickle
        self.x = pickle.loads(ob)


def worker(child_conn, parent_conn, make_env):
    parent_conn.close()
    # Create an environment
    env = make_env()
    
    while True:
        cmd, data = child_conn.recv()
        if cmd == 'step':
            obs, reward, done, info = env.step(data)
            if done:
                obs = env.reset()  # TODO: why reset
            child_conn.send([obs, reward, done, info])
        elif cmd == 'reset':
            obs = env.reset()
            child_conn.send(obs)
        elif cmd == 'reset_task':
            obs = env.reset_task()
            child_conn.send(obs)
        elif cmd == 'close':
            child_conn.close()
            break
        elif cmd == 'get_spaces':
            child_conn.send([env.observation_space, env.action_space])
        else:
            raise NotImplementedError

class SubprocVecEnv(VecEnv):
    """
    Run a list of environment in subprocesses
    """
    def __init__(self, list_make_env):
        self.waiting = False
        self.closed = False
        self.num_envs = len(list_make_env)
        self.parent_conns, self.child_conns = zip(*[Pipe() for _ in range(self.num_envs)])
        self.processes = []
        for parent_conn, child_conn, make_env in zip(self.parent_conns, self.child_conns, list_make_env):
            self.processes.append(Process(target=worker, args=[child_conn, parent_conn, CloudpickleWrapper(make_env)]))  # TODO: CloudpickleWrapper make_env
        for process in self.processes:
            process.daemon = True  # if the main process crashes, we should not cause things to hang
            process.start()
        for conn in self.child_conns:
            conn.close()
        
        # Obtain observation and action spaces
        self.parent_conns[0].send(['get_spaces', None])
        observation_space, action_space = self.parent_conns[0].recv()
        super().__init__(self.num_envs, observation_space, action_space)
        
    def step_asyn(self, actions):
        for parent_conn, action in zip(self.parent_conns, actions):
            parent_conn.send(['step', action])
            
        self.waiting = True
        
    def step_wait(self):
        observations, rewards, dones, infos = zip(*[parent_conn.recv() for parent_conn in self.parent_conns])
        self.waiting = False
        return np.stack(observations), np.stack(rewards), np.stack(dones), infos
    
    def reset(self):
        for parent_conn in self.parent_conns:
            parent_conn.send(['reset', None])
        return np.stack([parent_conn.recv() for parent_conn in self.parent_conns])
    
    def reset_task(self):
        for parent_conn in self.parent_conns:
            parent_conn.send(['reset_task', None])
        return np.stack([parent_conn.recv() for parent_conn in self.parent_conns])
    
    def close(self):
        if self.closed:
            return
        if self.waiting:
            for parent_conn in self.parent_conns:
                parent_conn.recv()
        for parent_conn in self.parent_conns:
            parent_conn.send(['close', None])
        for process in self.processes:
            process.join()
        self.closed = True

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    """
    Convolutional neural networks, from DeepMind Nature paper. 
    """
    def __init__(self, 
                 input_channel, 
                 conv_kernels,
                 conv_kernel_sizes,
                 conv_strides, 
                 conv_pads, 
                 conv_nonlinearity, 
                 hidden_sizes, 
                 hidden_nonlinearity, 
                 output_dim=None, 
                 output_nonlinearity=None):
        """
        Set up CNN with configurations. 
        
        Args:
            input_channel (int): the number of channels of the input, e.g. color channel
            conv_kernels (list): a list of number of kernels (filters or feature maps), for each convolutional layer. 
            conv_kernel_sizes (list): a list of kernel sizes, [int or tuple], for each convolutional layer. 
            conv_strides (list): a list of strides, for each convolutional layer. 
            conv_pads (list): a list of paddings, for each convolutional layer. 
            conv_nonlinearity (nn.functional): nonlinearity for convolutional layers
            hidden_sizes (list): a list of sizes for hidden layers
            hidden_nonlinearity (nn.functional): nonlinearity for hidden layers
            output_dim (int): output dimension
                                If None, then no output layer to be generated (useful if output has different heads)
            output_nonlinearity (nn.functional): nonlinearity for output layer
                                If None, then no output nonlinearity
        """
        super().__init__()
        
        self.input_channel = input_channel 
        self.conv_kernels = conv_kernels
        self.conv_kernel_sizes = conv_kernel_sizes
        self.conv_strides = conv_strides
        self.conv_pads = conv_pads 
        self.conv_nonlinearity = conv_nonlinearity
        self.hidden_sizes = hidden_sizes
        self.hidden_nonlinearity = hidden_nonlinearity
        self.output_dim = output_dim
        self.output_nonlinearity = output_nonlinearity
        
        

In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim


import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt


np.random.seed(2)

T = 20
L = 1000
N = 100

x = np.empty((N, L), 'int64')
x[:] = np.array(range(L)) + np.random.randint(-4 * T, 4 * T, N).reshape(N, 1)
data = np.sin(x/T).astype('float32')


class Sequence(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.lstm1 = nn.LSTMCell(1, 51)
        self.lstm2 = nn.LSTMCell(51, 51)
        self.linear = nn.Linear(51, 1)
        
    def forward(self, x, future=0):
        outputs = []
        batch_size, x_size = x.size()
        
        h_t = torch.zeros(batch_size, 51)
        c_t = torch.zeros(batch_size, 51)
        h_t_2 = torch.zeros(batch_size, 51)
        c_t_2 = torch.zeros(batch_size, 51)
        
        chunks = x.chunk(x_size, dim=1)
        
        for x_t in chunks:
            h_t, c_t = self.lstm1(x_t, (h_t, c_t))
            h_t_2, c_t_2 = self.lstm2(h_t, (h_t_2, c_t_2))
            
            output = self.linear(h_t_2)
            
            outputs.append(output)
            
        for _ in range(future):  # if we should predict the future
            h_t, c_t = self.lstm1(output, (h_t, c_t))
            h_t_2, c_t_2 = self.lstm2(h_t, (h_t_2, c_t_2))
            
            output = self.linear(h_t_2)
            
            outputs.append(output)
            
        outputs = torch.stack(outputs, dim=1).squeeze(2)
        
        return outputs
        
if __name__ == '__main__':

    np.random.seed(0)
    torch.manual_seed(0)

    input = torch.from_numpy(data[3:, :-1])
    target = torch.from_numpy(data[3:, 1:])
    test_input = torch.from_numpy(data[:3, :-1])
    test_target = torch.from_numpy(data[:3, 1:])

    seq = Sequence()

    criterion = nn.MSELoss()

    # Use LBFGS since we load whole data to train
    optimizer = optim.LBFGS(seq.parameters(), lr=0.8)

    for i in range(2):
        print(f'STEP: {i}')
        def closure():
            optimizer.zero_grad()
            out = seq(input)
            loss = criterion(out, target)
            print(f'Loss: {loss.item()}')
            loss.backward()
            return loss
        optimizer.step(closure)

        # Prediction
        future = 1000
        pred = seq(test_input, future=future)
        loss = criterion(pred[:, :-future], test_target)
        print(f'Test loss: {loss.item()}')
        y = pred.data.numpy()

        # Drawing
        plt.figure(figsize=(30, 10))
        plt.xlabel('x')
        plt.ylabel('y')
        plt.xticks(fontsize=20)
        plt.yticks(fontsize=20)
        def draw(yi, color):
            plt.plot(np.arange(input.size(1)), yi[:input.size(1)], color, linewidth=2.0)
            plt.plot(np.arange(input.size(1), input.size(1)+future), yi[input.size(1):], color + ':', linewidth=2.0)
        draw(y[0], 'r')
        draw(y[1], 'g')
        draw(y[2], 'b')
        
        plt.savefig(f'logs/{i}.pdf')